In [1]:
from FastRCNN.data_setup import create_dataloaders
from torchvision.transforms import v2 as T
import torch

def get_transform(train):
    transforms = []
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ToDtype(torch.float, scale=True))
    transforms.append(T.ToPureTensor())
    return T.Compose(transforms)

train_dir = "data/train"
train_coco = "data/annotations/train.json"
val_dir = "data/val"
val_coco = "data/annotations/val.json"
test_dir = "data/test"
test_coco = "data/annotations/test.json"
train_dataloader, val_dataloader, test_dataloader = create_dataloaders(train_dir,
                                                                       val_dir,
                                                                       test_dir,
                                                                       train_coco,
                                                                       val_coco,
                                                                       test_coco,
                                                                       get_transform(False),
                                                                       1,
                                                                       0)

d:\Coding Applications\miniconda\envs\fyp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [2]:
from FastRCNN.engine import train_one_epoch, evaluate
from FastRCNN.model_builder import get_model_instance_segmentation
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
fastRCNN_model = get_model_instance_segmentation(4).to(device)
print(device)

cuda


In [3]:
params = [p for p in fastRCNN_model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

num_epochs = 2

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(fastRCNN_model, optimizer, train_dataloader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(fastRCNN_model, test_dataloader, device=device)

print("That's it!")

Epoch: [0]  [   0/1959]  eta: 1:07:34  lr: 0.000010  loss: 4.4935 (4.4935)  loss_classifier: 1.8825 (1.8825)  loss_box_reg: 0.1670 (0.1670)  loss_mask: 2.4173 (2.4173)  loss_objectness: 0.0260 (0.0260)  loss_rpn_box_reg: 0.0007 (0.0007)  time: 2.0696  data: 0.0044  max mem: 905


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
